# Linear Programming Project 

## Problem description
There are 3 delivery depots in Vancouver, and need to delivery packages to 5 different areas, different areas have different demands of packages. How the delivery company assign their deliverers to different depots such that this optimal strategy can minimize the cost.

We simply consider all depots and delivery areas as nodes, and the connection between depots and delivery areas as roads, building a graph-based model. We first formulat a linear programming problme to solve a solution for best strategy to assign the number of delivery trips on each roads between depots and delivery areas such that the cost on runing the trips (the salary we should pay for the distance the delivery travel). 

Next, we base on the optimal solutions of trips assignment stategy build a new linear programming problem and solve it for the best choice to assign the number of works to each depost.


## Variables and Parameters

## Object function